In [4]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from operator import add
import math

In [2]:
# !pip install findspark

In [3]:
#Creating Spark Context
sc = SparkContext.getOrCreate()
#Creating Spark Session
spark = SparkSession(sc)
#Readind data set(csv file),dropping null values and selecting only tabs and views column
tags_views = spark.read.csv("USvideos.csv", inferSchema = True, header = True).dropna().select("tags","views")
#Mapping by splitting tags with "|" character,
def tags_split(x):
    tags=x["tags"].split("|")
    result=[]

In [2]:
# !pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 52.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=21ccf0f9fc815c03f726e85f64aea2cc45d3a99dcdc3f51770db103709d2ba31
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
def tags_split(x):
    tags=x["tags"].split("|")
    result=[]
    for every in tags:
            #Associating view count as counter and applying log on view count because views count will be really large
            if not x["views"].isdigit() or every==None:
                continue        
            result.append((every.strip("\"").lower(),math.log(int(x["views"]))))#Stripping unnecessary characters
    return tuple(result)
rdd1=tags_views.rdd.flatMap(tags_split).reduceByKey(add)#reduce by similar tags and adding its view count
#Top Tags are queried from RDD by Sorting RDD's in descending order of view count
toptags=rdd1.takeOrdered(35, key = lambda x: -x[1])
df=spark.createDataFrame(toptags)
#Writing back to Disk
df.repartition(1).write.csv(path="file:\D\trending.csv")
# df.repartition(1).write.csv(path="file:///C:\\Users\\saikr\\Desktop\\DIC\\trending.csv")
#Spark session
spark.stop()


NameError: name 'tags_views' is not defined